In [1]:
class Opion():
    
    def __init__(self):

        self.dataroot= r'/home/jara/DeepInPainting/paris_original_test' #image dataroot
        self.maskroot= r'/home/jara/DeepInPainting/mask/test_mask' #mask dataroot
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# second stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'/home/jara/DeepInPainting/checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=60
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=False



In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
from IQA_pytorch import SSIM
import numpy as np
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [
     transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])



dataset_test = Data_load(opt.dataroot, opt.maskroot, transform, transform_mask)
iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=False))
print(len(dataset_test))
model = create_model(opt)
total_steps = 0


2400
csa_net
initialize network with normal


/home/jara/DeepInPainting/models/networks.py:53: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
/home/jara/DeepInPainting/models/networks.py:63: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
Loading pre-trained network!
model [CSAModel] was created


In [3]:
load_epoch=60
model.load(load_epoch)

In [ ]:
save_dir = r'/home/jara/DeepInPainting/result'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
psnr_average, ssim_average = 0, 0
psnr_sum, ssim_sum = 0, 0
for image, mask in (iterator_test):
    
    if epoch > 500:
        psnr_average = psnr_sum / 500
        ssim_average = ssim_sum / 500
        print("Finish")
        print('PSNR_average : %.2f, SSIM_average : %.3f' %(psnr_average, ssim_average))
        break

    iter_start_time = time.time()
    image=image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.bool()
    
    model.set_input(image,mask)
    model.set_gt_latent()
    model.test()
    
    real_A,real_B,fake_B,fake_P=model.get_current_visuals()
    pic = (torch.cat([real_A, real_B,fake_P,fake_B], dim=0) + 1) / 2.0
    
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
    save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=2)
    
    psnr=0
    mse=0
    if type(real_B)==torch.Tensor:
        mse=torch.mean((real_B-fake_B)**2)
    else:
        mse=np.mean((real_B-fake_B)**2)
    
    if mse==0:
        psnr=100
    else:
        psnr=10*torch.log10((2**2)/mse)
    errors=model.get_error()
    
    mod=SSIM(channels=3)
    score=mod(real_B,fake_B,as_loss=False)
    print('%d. PSNR : %f, SSIM : %f' %
         (epoch,psnr,score))
    psnr_sum += psnr
    ssim_sum += score
    epoch=epoch+1

1. PSNR : 23.041302, SSIM : 0.719469
2. PSNR : 25.936487, SSIM : 0.755162
3. PSNR : 26.033058, SSIM : 0.875370
4. PSNR : 17.966557, SSIM : 0.586449
5. PSNR : 21.245298, SSIM : 0.737485
6. PSNR : 27.909550, SSIM : 0.826507
7. PSNR : 15.185556, SSIM : 0.484679
8. PSNR : 23.669130, SSIM : 0.715878
9. PSNR : 31.639593, SSIM : 0.935556
10. PSNR : 31.125828, SSIM : 0.911767
11. PSNR : 19.020826, SSIM : 0.557828
12. PSNR : 22.003222, SSIM : 0.700762
13. PSNR : 27.086460, SSIM : 0.887183
14. PSNR : 37.269539, SSIM : 0.985178
15. PSNR : 33.490196, SSIM : 0.947635
16. PSNR : 31.442894, SSIM : 0.950130
17. PSNR : 20.435238, SSIM : 0.678795
18. PSNR : 27.024696, SSIM : 0.898192
19. PSNR : 14.876327, SSIM : 0.449718
20. PSNR : 17.380001, SSIM : 0.721736
21. PSNR : 32.506687, SSIM : 0.966909
22. PSNR : 32.577732, SSIM : 0.909507
23. PSNR : 24.502172, SSIM : 0.901587
24. PSNR : 26.137087, SSIM : 0.812152
25. PSNR : 23.332300, SSIM : 0.678369
26. PSNR : 18.216442, SSIM : 0.499556
27. PSNR : 22.811066,